In [429]:
import random

class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit  # gold, silver, bronze, black, special

    def __repr__(self):
        return f"{self.value} of {self.suit}"

def create_deck():
    suits = ['gold', 'silver', 'bronze', 'black']
    special_suits = ['special']  # For multiplication and square root cards
    deck = [Card(value, suit) for value in range(0, 11) for suit in suits]
    deck += [Card('*', special_suit) for special_suit in special_suits for _ in range(4)]
    deck += [Card('√', special_suit) for special_suit in special_suits for _ in range(4)]
    random.shuffle(deck)
    return deck

def initialize_players():
    player_count = int(input("Enter the number of players: "))
    players = [Player(f"Player {i+1}") for i in range(player_count)]
    return players

def suit_hierarchy(suit):
    # convert suit to int
    hierarchy = {'gold': 4, 'silver': 3, 'bronze': 2, 'black': 1}
    return hierarchy.get(suit, 0)

In [430]:
class Player:
    def __init__(self, name):
        self.name = name
        self.chips = 50
        self.hand = []
        self.operators = ['+', '-', '/']
        self.hidden_card = None
        self.bet = 0
        self.equation = ''
        self.fold = False
        self.current_bet = 0
        self.bet_type = ''
        self.equation_result = None

    def receive_card(self, card):
        if isinstance(card.value, int):
            self.hand.append(card)
        else:
            self.operators.append(card.value)

    def set_hidden_card(self, card):
        self.hidden_card = card

    def show_hand(self):
        return ', '.join(str(card) for card in self.hand)
    
    def show_all_cards(self):
        all_cards = ', '.join(str(card) for card in self.hand)
        hidden_card_str = f"Hidden Card: {self.hidden_card}" if self.hidden_card else "No Hidden Card"
        operators_str = "Operators: " + ', '.join(self.operators)
        return f"Open Cards: {all_cards}; {hidden_card_str}; {operators_str}"
    
    def does_have_equation(self, equation):
        # Split the equation into components
        tokens = re.findall(r'\d+|\D', equation)
        numbers = [int(token) for token in tokens if token.isdigit()]
        operators = [token for token in tokens if not token.isdigit()]

        # Create a dictionary to track the occurrences of each card in the equation
        num_occurence_equation = {number: 0 for number in numbers}
        for number in numbers:
            if number in num_occurence_equation:
                num_occurence_equation[number] += 1
                
        op_occurence_equation = {op: 0 for op in operators}
        for op in operators:
            if op in op_occurence_equation:
                op_occurence_equation[op] += 1

        # Check if all numbers are in the player's hand and not used more than available
        whole_hand = self.hand
        whole_hand.append(self.hidden_card)
        num_occurence_hand = [card.value for card in whole_hand]
        for number in numbers:
            if num_occurence_equation[number] > num_occurence_hand.count(number):
                print("Invalid equation: used number not in hand or used more than available")
                return False

        # Check if all operators are in the player's hand and not used more than available
        op_occurence_hand = {op: 0 for op in self.operators}
        for op in self.operators:
            if op in op_occurence_hand:
                op_occurence_hand[op] += 1
        
        for op in operators:
            try:
                if op_occurence_equation[op] != op_occurence_hand[op]:
                    print(f"Invalid equation: used operator {op} not in hand")
                    return False
            except KeyError:
                print(f"Invalid equation: used operator {op} not in hand")
                return False
            
        return True

    def highest_card_info(self):
        # Returns the highest card's numeric value and suit hierarchy
        if not self.hand:
            return (0, 0)
        highest_card = max(self.hand, key=lambda card: (card.value, suit_hierarchy(card.suit)))
        return (highest_card.value, suit_hierarchy(highest_card.suit))

    def lowest_card_info(self):
        # Returns the lowest card's numeric value and suit hierarchy
        numeric_cards = [card for card in self.hand if isinstance(card.value, int)]
        if not numeric_cards:
            return (float('inf'), 0)
        lowest_card = min(numeric_cards, key=lambda card: (card.value, suit_hierarchy(card.suit)))
        return (lowest_card.value, suit_hierarchy(lowest_card.suit))

In [356]:
def betting_round(players, starting_player, mandatory_bet, allow_multiple_raises=True):
    round_bet = 0 # total bet for this whole round per player
    round_total_bet = 0 # total bet from all players
    
    last_raiser = None
    current_player = starting_player

    # Initial mandatory bet
    if mandatory_bet != 0:
        for i in range(len(players)):
            player = players[(starting_player + i) % len(players)]
            if player.chips > 0:
                bet_amount = min(mandatory_bet, player.chips)
                player.chips -= bet_amount
                player.current_bet = bet_amount
                round_total_bet += bet_amount
                print(f"{player.name} posts mandatory bet of {bet_amount} chips.")
        round_bet += mandatory_bet

    while True:
        player = players[current_player % len(players)]
        current_player += 1

        # check if dead
        if player.chips == 0 or player.fold:
            if current_player % len(players) == starting_player:
                break
            continue

        # Print info for the player
        print(f"{player.name}'s turn. You've bet {player.current_bet} this round. Current bet is now: {round_bet} chips")
        print(f"Your chips: {player.chips}.")
        print('Your cards: ', player.show_all_cards())
        action = input("Choose action (raise/call/check/fold): ").lower()

        if action == 'raise':
            if last_raiser == player and not allow_multiple_raises:
                print("Multiple raises are not allowed. Choose another action.")
                current_player -= 1
                continue
            while True:
                bet_amount = int(input("Enter bet amount: "))
                if bet_amount <= (round_bet - player.current_bet) or bet_amount > player.chips:
                    print('You need to bet at least the current bet:', round_bet, 'and max', player.chips, 'chips')
                else:
                    break
            player.chips -= bet_amount
            player.current_bet += bet_amount
            round_bet += bet_amount
            round_total_bet += bet_amount
            last_raiser = player
            
        elif action == 'call':
            call_amount = round_bet - player.current_bet
            player.chips -= call_amount
            player.current_bet += call_amount
            round_total_bet += call_amount
            
        elif action == 'check':
            if round_bet != player.current_bet:
                print("Cannot check. There's a bet already. You need to call, raise, or fold.")
                current_player -= 1
                continue
        elif action == 'fold':
            player.fold = True
            print(f"{player.name} folded.")

        print(f"{player.name} now has {player.chips} chips.")

        if all(p.fold or p.chips == 0 or p.current_bet == round_bet for p in players):
            if current_player % len(players) == starting_player:
                break

    # Reset player bets and folds for the next round
    for player in players:
        player.current_bet = 0
        # player.fold = False

    return round_total_bet

In [357]:
def deal_initial_cards(players, deck):
    for player in players:
        if player.hand != []:
            raise Exception("To deal again initial cards, initiate the players again.") 
        # Deal the hidden card
        # If the hidden card is × or √, return it to the deck, reshuffle, and deal new card
        hidden_card = deck.pop()
        while hidden_card.value in ['*', '√']:
            deck.append(hidden_card)
            random.shuffle(deck)
            hidden_card = deck.pop()
        player.set_hidden_card(hidden_card)

        # Deal two open cards
        # If √ ==> add number card
        # If × ==> discard operator card except /
        for _ in range(2):
            card = deal_open_card(player, deck)
            player.receive_card(card)

def deal_open_card(player, deck):
    card = deck.pop()
    if card.value == '*':
        print(f'{player.name}, you have got * operator.')
        discard_operator_card(player)
        player.receive_card(card)
        card = deal_number_card(player, deck)
    elif card.value == '√':
        print(f'{player.name}, you have got √ operator and you will recieve one additional number card.')
        player.receive_card(card)
        card = deal_number_card(player, deck)
    return card

def deal_number_card(player, deck):
    while True:
        card = deck.pop()
        if card.value not in ['*', '√']:
            print(f'{player.name}, you recieve {card}')
            return card

def discard_operator_card(player):
    print(f"{player.name}, please choose an operator to discard from {player.operators} except '/': ")
    operator = input()
    while operator not in player.operators or operator == '/':
        print("Invalid operator. Choose again: ")
        operator = input()
    player.operators.remove(str(operator))

def deal_additional_card(players, deck):
    for player in players:
        card = deal_open_card(player, deck)
        player.receive_card(card)


In [423]:
import re
import ast
import operator

ops = {
    ast.Add: operator.add, 
    ast.Sub: operator.sub, 
    ast.Mult: operator.mul, 
    ast.Div: operator.truediv
}

class EvalVisitor(ast.NodeVisitor):
    def visit_BinOp(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        op_type = type(node.op)
        if op_type in ops:
            return ops[op_type](left, right)
        raise ValueError(f"Unsupported operation {node.op}")

    def visit_Num(self, node):
        return node.n

def evaluate_equation(equation):
    if '√' in equation:
        equation = re.sub(r'√(\d+)', r'(\1**0.5)', equation)
        
    try:
        tree = ast.parse(equation, mode='eval')
        ev = EvalVisitor()
        return ev.visit(tree.body)
    except Exception as e:
        print(f"Error evaluating equation: {e}")
        return None

def process_equation(player, equation):
    if not re.fullmatch(r'[0-9+\-*/√\s]+', equation):
        return 'Invalid characters in expression'

    if player.does_have_equation(equation):
        return evaluate_equation(equation)
    else:
        return 'Equation uses cards not in hand or uses cards more than available'
    # add rule to use all numbers and operators


def player_make_equation(player):
    while True:
        print(f"{player.name}, your cards: {player.show_all_cards()}")
        equation = input("Enter your equation: ")
        result = process_equation(player, equation)

        if isinstance(result, str):
            print('co jsem timto zamyslel?', result)  # Error message
        else:
            print(f"Equation result: {result}")
            return result

In [431]:
def high_or_low(players):
    for player in players:
        while True:
            # can be omitted since I input only active players
            if player.fold or player.chips == 0:
                break
            bet_type = input(f'{player.name}, do you bet on low(1) or high(20)? ').lower()
            if bet_type in ['low', 'high']:
                player.bet_type = bet_type
                break
            else:
                print('Invalid input. Please write "low" or "high".')

def determine_winners(players, total_bet):
    high_players = [p for p in players if p.bet_type == 'high']
    low_players = [p for p in players if p.bet_type == 'low']

    high_winner = None
    low_winner = None

    if high_players:
        high_winner = max(high_players, key=lambda p: -abs(p.equation_result - 20))
        # Check for tie in high pot
        if len(set(-abs(p.equation_result - 20) for p in high_players)) == 1:
            high_winner = break_tie(high_players, 'high')
        
    if low_players:
        low_winner = min(low_players, key=lambda p: abs(p.equation_result - 1))
        # Check for tie in low pot
        if len(set(abs(p.equation_result - 1) for p in low_players)) == 1:
            low_winner = break_tie(low_players, 'low')

    # Determine the winning amounts
    if high_winner and low_winner:
        winning_amount = total_bet / 2
        print(f"{high_winner.name} wins {winning_amount} chips from high pot.")
        print(f"{low_winner.name} wins {winning_amount} chips from low pot.")
    elif high_winner:
        print(f"{high_winner.name} wins {total_bet} chips as the sole winner in high pot.")
    elif low_winner:
        print(f"{low_winner.name} wins {total_bet} chips as the sole winner in low pot.")

    return high_winner, low_winner

def break_tie(players, pot_type):
    if pot_type == 'high':
        # Sort players by their highest card info (number and color)
        sorted_players = sorted(players, key=lambda p: p.highest_card_info(), reverse=True)
    else:  # 'low' pot
        # Sort players by their lowest card info (number and color)
        sorted_players = sorted(players, key=lambda p: p.lowest_card_info())

    # The first player in the sorted list is the winner
    return sorted_players[0]


def end_game(players):
    for player in players:
        # pieces = player.chips // 5
        print(f"{player.name} ends the game with {player.chips} chips.")

In [425]:
def main():
    players = initialize_players()
    starting_player = 0
    mandatory_bet = 5   
    
    while len(players) >= 1:
        
        deck = create_deck()
        deal_initial_cards(players, deck)
        # TODO...print cards...


        # First betting round
        total_bet = betting_round(players, starting_player, mandatory_bet)

        print('-----Next round, dealing one more card.-----')
        # only for player who did not fold
        playing_players = [p for p in players if p.fold == False]
        deal_additional_card(playing_players, deck)

        # Second betting round
        print('-----Second betting round-----')
        total_bet += betting_round(players, starting_player, mandatory_bet=0)

        # update playing players for this round
        playing_players = [p for p in players if p.fold == False]
        # Players choose high or low
        high_or_low(playing_players)

        # Players make their equations
        for player in playing_players:
            player.equation_result = player_make_equation(player)

        # Determine winners
        high_winner, low_winner = determine_winners(playing_players, total_bet)

        # End of game round
        end_game(players)
        
        # Reset fold:
        for player in players:
            player.fold = False

        # Next game starts the player on the right
        starting_player = (starting_player + 1) % len(players)
        
        # remove players with 0 chips from the whole game
        players = [player for player in players if player.chips > 0]

if __name__ == "__main__":
    main()
# enter equation for each player immediately after high,low
# chytat kdyz spatnej input "action" 
# nejak to zase nefugnuje check a furt se hraje i kdyz vsichni check
# try: 1) tie mechanism (not tested)
#      2) multiple game rounds + death (0 chips)

Enter the number of players: 2
Player 1 posts mandatory bet of 5 chips.
Player 2 posts mandatory bet of 5 chips.
Player 1's turn. You've bet 5 this round. Current bet is now: 5 chips
Your chips: 45.
Your cards:  Open Cards: 0 of gold, 8 of gold; Hidden Card: 2 of black; Operators: +, -, /
Choose action (raise/call/check/fold): check
Player 1 now has 45 chips.
Player 2's turn. You've bet 5 this round. Current bet is now: 5 chips
Your chips: 45.
Your cards:  Open Cards: 6 of black, 3 of black; Hidden Card: 0 of silver; Operators: +, -, /
Choose action (raise/call/check/fold): check
Player 2 now has 45 chips.
-----Next round, dealing one more card.-----
-----Second betting round-----
Player 1's turn. You've bet 0 this round. Current bet is now: 0 chips
Your chips: 45.
Your cards:  Open Cards: 0 of gold, 8 of gold, 8 of bronze; Hidden Card: 2 of black; Operators: +, -, /
Choose action (raise/call/check/fold): check
Player 1 now has 45 chips.
Player 2's turn. You've bet 0 this round. Curren